In [1]:
import torch

import nbimporter
from preprocFinalExperiment import *

Importing Jupyter notebook from preprocFinalExperiment.ipynb
Importing Jupyter notebook from preprocVisualisationTesting.ipynb


In [ ]:
dataStats = getDataStatistics(train_x, train_y)

In [ ]:
dataStats

In [2]:
import plotly
from plotly.offline import iplot as plt
from plotly import graph_objs as plt_type
plotly.offline.init_notebook_mode(connected=True)

import nbimporter
from preprocVisualisationTesting import *

In [ ]:
import torch
import torch.

In [ ]:
(train_y < 0.).sum().float()/float(train_y.numel())

In [7]:
for dataset_name in ['neurofinder.00.00', 'neurofinder.01.00', 'neurofinder.02.00', 'neurofinder.03.00', 'neurofinder.04.00']:
    trainingData = extractTrainingData(dataset_name, max_T=500, data_dir='/nfs/data3/gergo/Neurofinder_update/', 
                            remove_PCs = None,
                            stamp='')# , force_redo=False)

    input_device = 'cpu'
    train_x = trainingData['train_x'].to(input_device).float()
    train_y = trainingData['train_y'].to(input_device)
    mean_im = trainingData['mean_im'].to(input_device)

    tmp = train_y.contiguous().view(-1).cpu()
    hist, bins = np.histogram(tmp[tmp<np.quantile(tmp, 0.4)], bins=100)

    hist = torch.tensor(hist).to(input_device)
    bins = torch.tensor(bins).to(input_device)

    bin_size = bins[1] - bins[0]
    filter_size = torch.tensor(int(11)).to(input_device) #(5./bin_size).round().int()

    filter_weights = torch.distributions.Normal(loc=filter_size.float()/2., scale=filter_size.float()/4.
                                               ).log_prob(torch.arange(filter_size).float().to(input_device)
                                                         ).exp().to(input_device)
    filter_weights = filter_weights.div(filter_weights.sum()).to(input_device)

    # Gaussian smoothing of histograms
    hist_smooth = torch.nn.functional.conv1d(
        hist.view(1,1,-1).float(), 
        filter_weights.view(1,1,-1), 
        padding=int((filter_size-1)/2))
    
    # Correction for edge effect
    corr_smooth = torch.nn.functional.conv1d(
        torch.ones_like(hist.view(1,1,-1).float()), 
        filter_weights.view(1,1,-1), 
        padding=int((filter_size-1)/2))
    
    hist_smooth = hist_smooth.div(corr_smooth).squeeze()
    
    # Find the peak and the width (to the right) of the peak
    out = OrderedDict()
    out['y_pedestal_loc'] = bins[hist_smooth.argmax()].to(input_device)
    out['y_pedestal_scale'] = (bins[
        (hist_smooth[hist_smooth.argmax():]-(hist_smooth.max()-hist_smooth.min())/2).abs().argmin() 
        + hist_smooth.argmax()]
             - out['y_pedestal_loc']).to(input_device)
    
    print(out)
    import math
    print(filter_size.float()/4.*bin_size, (out['y_pedestal_scale']/math.sqrt(2.*math.log(2.)))) 
    print(((out['y_pedestal_scale']/math.sqrt(2.*math.log(2.))).pow(2) - (filter_size.float()/4.*bin_size).pow(2)).sqrt())

    plt([plt_type.Bar(x=bins, y=hist),
         plt_type.Scatter(x=bins, y=hist_smooth.squeeze())])

OrderedDict([('y_pedestal_loc', tensor(7.5200)), ('y_pedestal_scale', tensor(4.2300))])
tensor(1.2925) tensor(3.5926)
tensor(3.3521)


OrderedDict([('y_pedestal_loc', tensor(102.8038)), ('y_pedestal_scale', tensor(67.2898))])
tensor(5.1402) tensor(57.1507)
tensor(56.9191)


OrderedDict([('y_pedestal_loc', tensor(0.)), ('y_pedestal_scale', tensor(0.7500))])
tensor(0.6875) tensor(0.6370)
tensor(nan)


OrderedDict([('y_pedestal_loc', tensor(0.)), ('y_pedestal_scale', tensor(78.7980))])
tensor(43.3389) tensor(66.9249)
tensor(50.9968)


OrderedDict([('y_pedestal_loc', tensor(0.)), ('y_pedestal_scale', tensor(5.9940))])
tensor(1.0989) tensor(5.0908)
tensor(4.9708)


In [ ]:
hist_smooth.argmax()

In [ ]:
hist_smooth.shape

In [ ]:
bins[hist_smooth.argmax()]

In [ ]:
filter_weights

In [ ]:
from collections import OrderedDict
import warnings
import numpy as np

from preprocUtils import *

def getDataStatistics(train_x, train_y=None, expected_latent_photon_count_guess = 2.0):
    """
    Given training data, and expected true photon counts before gain modulation,
    return some data statistics that may be used to initialise
    usual parameters such as
        - lengthscales
        - output scales
        - gains
        - expected photon counts
    """
    input_device = train_x.device
    
    out = OrderedDict()
    
    out['expected_latent_photon_count'] = expected_latent_photon_count_guess
    
    x_minmax = torch.stack([train_x.min(0)[0], train_x.max(0)[0]])
    
    out['x_minmax'] = x_minmax
    out['x_span'] = x_minmax.unbind(1) # list of dimension-wise bounds tensors of train_x
    out['x_width'] = x_minmax[1,:] - x_minmax[0,:]
    out['x_std'] = train_x.std(0)
    out['n_data'] = torch.tensor(train_x.size(0), device=input_device).float()
    out['n_data_per_dim'] = out['n_data'].pow(1./train_x.size(1))
    
    # For lengthscales combine x_width with x_std or n_data_per_dim
    out['lengthscale_guess'] = out['x_width']/out['n_data_per_dim']*2.0
    
    
    if train_y is not None:
        # Estimate of photomultiplier linear gain
        # Do affine linear regression from train_y.mean(1) to train_y.var(1)
        predictor = train_y.mean(1).unsqueeze(-1)
        target = train_y.var(1).unsqueeze(-1)
        out['y_gain_linear'], tmp, out['y_gain_offset'] = (
            torchLinReg(train_y.mean(1).unsqueeze(-1), train_y.var(1).unsqueeze(-1)))

        # Make a guess about the expected latent output scale by taking the spatial std after gain-correction
        out['latent_output_scale'] = predictor.squeeze().div(out['y_gain_linear']).std()
        
        
        # Estimate the pedestal peak position and scale
        
        

        #out['y_gain'] = train_y.median().div(expected_latent_photon_count_guess)
    
    
    return out